In [1]:
    # Given requirements data
    requirements_data = [
        {
            "index": "1",
            "title": "Safety Feature Requirements",
            "description": "The garage door opener must have a safety feature to prevent the door from closing if there is an obstacle in its path or if someone is under the door."
        },
        {
            "index": "2",
            "title": "User Interface Requirements",
            "description": "The garage door opener must have an intuitive user interface that allows users to easily open, close, and control the garage door remotely using a remote control, smartphone app, or wall switch."
        },
        {
            "index": "3",
            "title": "Durability Requirements",
            "description": "The garage door opener must be designed to withstand harsh weather conditions, including extreme temperatures, humidity, and precipitation."
        },
        {
            "index": "4",
            "title": "Security Requirements",
            "description": "The garage door opener must have advanced security features to prevent unauthorized access and hacking attempts, including encryption, password protection, and secure communication protocols."
        },
        {
            "index": "5",
            "title": "Energy Efficiency Requirements",
            "description": "The garage door opener must be energy-efficient and designed to minimize energy consumption, including features such as automatic door reversal, soft start, and low-power modes."
        },
        {
            "index": "6",
            "title": "Noise Reduction Requirements",
            "description": "The garage door opener must be designed to minimize noise levels, including features such as reduced motor noise, silent belt drive, and sound-absorbing materials."
        },
        {
            "index": "7",
            "title": "Maintenance Requirements",
            "description": "The garage door opener must be designed for easy maintenance and repair, including features such as accessible components, clear documentation, and online support."
        },
        {
            "index": "8",
            "title": "Compatibility Requirements",
            "description": "The garage door opener must be compatible with a wide range of garage door sizes and materials, including steel, wood, and aluminum doors."
        },
        {
            "index": "9",
            "title": "Certification Requirements",
            "description": "The garage door opener must meet or exceed all relevant industry standards and certifications, including UL, ETL, and CE."
        },
        {
            "index": "10",
            "title": "Warranty Requirements",
            "description": "The garage door opener must come with a comprehensive warranty that covers parts, labor, and repairs for a minimum of 5 years."
        }
    ]


In [3]:
import json

class StateMemory:
    def __init__(self):
        self.components = {}
        self.requirements = {}
        self.functions = {}
        self.physicals = {}
        self.top_component_id = None  # Track the ID of the top-level component

    # Add a new component
    def add_component(self, name, description=None):
        if self.top_component_id is not None:
            raise ValueError("There can only be one top-level component.")

        component = Component(name, description)
        self.components[component.id] = component
        self.top_component_id = component.id  # Update the top_component_id
        return component

    # Get the top-level component
    def get_top_component(self):
        if not self.top_component_id:
            raise ValueError("No top-level component exists.")
        return self.components[self.top_component_id]

    # Get the state as a hierarchical JSON structure
    def get_state_as_json(self):
        if not self.top_component_id:
            return json.dumps({"error": "No top-level component exists."}, indent=2)

        def build_hierarchy(component_id):
            component = self.components[component_id]
            return {
                "id": component.id,
                "name": component.name,
                "description": component.description,
                "requirements": [
                    {
                        "id": req.id,
                        "name": req.name,
                        "description": req.description,
                        "functions": [
                            {
                                "id": func.id,
                                "name": func.name,
                                "description": func.description,
                                "physicals": [
                                    {
                                        "id": phys.id,
                                        "name": phys.name,
                                        "description": phys.description,
                                    }
                                    for phys in self.physicals.values()
                                    if phys.parent_id == func.id
                                ],
                            }
                            for func in self.functions.values()
                            if func.parent_id == req.id
                        ],
                    }
                    for req in self.requirements.values()
                    if req.parent_id == component_id
                ],
            }

        hierarchy = build_hierarchy(self.top_component_id)
        return json.dumps(hierarchy, indent=2)

# Component, Requirement, Function, and Physical classes for the structure
import uuid

class Component:
    def __init__(self, name, description=None):
        self.id = str(uuid.uuid4())
        self.name = name
        self.description = description

class Requirement:
    def __init__(self, name, description=None, parent_id=None):
        self.id = str(uuid.uuid4())
        self.name = name
        self.description = description
        self.parent_id = parent_id

class Function:
    def __init__(self, name, description=None, parent_id=None):
        self.id = str(uuid.uuid4())
        self.name = name
        self.description = description
        self.parent_id = parent_id

class Physical:
    def __init__(self, name, description=None, parent_id=None):
        self.id = str(uuid.uuid4())
        self.name = name
        self.description = description
        self.parent_id = parent_id


In [6]:
import concurrent.futures
import time

class LLMProcessorParallel:
    def __init__(self, llm_client, state_memory):
        self.llm_client = llm_client
        self.state_memory = state_memory

    def create_physical_for_requirements_parallel(self, requirements_data):
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Map each requirement to a separate thread
            futures = [
                executor.submit(self.process_requirement, requirement)
                for requirement in requirements_data
            ]
            
            for future in concurrent.futures.as_completed(futures):
                try:
                    result = future.result()  # Get the result (if needed)
                    print(f"Finished processing: {result}")
                except Exception as e:
                    print(f"An error occurred: {e}")

    def process_requirement(self, requirement):
        index = requirement["index"]
        title = requirement["title"]
        description = requirement["description"]

        print(f"Processing requirement {index}: {title}")

        # Create a requirement object in the state memory
        req = Requirement(name=title, description=description)
        self.state_memory.requirements[req.id] = req

        # Call the LLM to create a physical for this requirement
        physical_data = self.call_llm_to_create_physical(req)

        # Create a physical object in the state memory
        physical = Physical(
            name=physical_data["name"],
            description=physical_data["description"],
            parent_id=req.id
        )
        self.state_memory.physicals[physical.id] = physical

        return f"Physical created for requirement {index}: {physical.name}"

    def call_llm_to_create_physical(self, requirement):
        """
        Calls the LLM to generate a 'physical' for the given requirement.
        Replace this with the OpenAI API call in production.
        """
        # Example LLM prompt
        prompt = f"Create a physical design for the following requirement:\n\n{requirement.description}\n"

        # Simulating an LLM call response
        response = self.llm_client.generate(prompt)

        # Example structure of the LLM response
        return {
            "name": f"Physical for {requirement.name}",
            "description": response
        }

# Simulated OpenAI LLM client
class MockOpenAIClient:
    def generate(self, prompt):
        time.sleep(2)  # Simulate API processing time
        return f"Generated design based on the requirement: {prompt[:50]}..."

# Example usage
if __name__ == "__main__":
    # Initialize the mock OpenAI client and state memory
    llm_client = MockOpenAIClient()
    state_memory = StateMemory()

    # Initialize the LLM processor for parallel processing
    llm_processor_parallel = LLMProcessorParallel(llm_client, state_memory)

    # Call the function to create physicals for the given requirements in parallel
    llm_processor_parallel.create_physical_for_requirements_parallel(requirements_data)

    # Output the state as JSON
    print("\nFinal State in JSON:")
    print(state_memory.get_state_as_json())


Processing requirement 1: Safety Feature Requirements
Processing requirement 2: User Interface Requirements
Processing requirement 3: Durability Requirements
Processing requirement 4: Security Requirements
Processing requirement 5: Energy Efficiency Requirements
Processing requirement 6: Noise Reduction Requirements
Processing requirement 7: Maintenance Requirements
Processing requirement 8: Compatibility Requirements
Processing requirement 9: Certification Requirements
Processing requirement 10: Warranty Requirements
Finished processing: Physical created for requirement 1: Physical for Safety Feature Requirements
Finished processing: Physical created for requirement 2: Physical for User Interface Requirements
Finished processing: Physical created for requirement 3: Physical for Durability Requirements
Finished processing: Physical created for requirement 4: Physical for Security Requirements
Finished processing: Physical created for requirement 5: Physical for Energy Efficiency Requir

# using netvibes llm - Parallel processing

In [5]:
import concurrent.futures
import time
import os
from langchain_openai import AzureChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

class LLMProcessorParallel:
    def __init__(self, llm_client, state_memory):
        self.llm_client = llm_client
        self.state_memory = state_memory

    def create_physical_for_requirements_parallel(self, requirements_data):
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Map each requirement to a separate thread
            futures = [
                executor.submit(self.process_requirement, requirement)
                for requirement in requirements_data
            ]
            
            for future in concurrent.futures.as_completed(futures):
                try:
                    result = future.result()  # Get the result (if needed)
                    print(f"Finished processing: {result}")
                except Exception as e:
                    print(f"An error occurred: {e}")

    def process_requirement(self, requirement):
        index = requirement["index"]
        title = requirement["title"]
        description = requirement["description"]

        print(f"Processing requirement {index}: {title}")

        # Create a requirement object in the state memory
        req = Requirement(name=title, description=description)
        self.state_memory.requirements[req.id] = req

        # Call the LLM to create a physical for this requirement
        physical_data = self.call_llm_to_create_physical(req)

        # Create a physical object in the state memory
        physical = Physical(
            name=physical_data["name"],
            description=physical_data["description"],
            parent_id=req.id
        )
        self.state_memory.physicals[physical.id] = physical

        return f"Physical created for requirement {index}: {physical.name}"

    def call_llm_to_create_physical(self, requirement):
        """
        Calls the LLM to generate a 'physical' for the given requirement.
        """
        # Example LLM prompt
        prompt = f"Create a physical design for the following requirement:\n\n{requirement.description}\n"

        # Call the LLM and get the response
        response = self.llm_client(prompt)

        # Example structure of the LLM response
        return {
            "name": f"Physical for {requirement.name}",
            "description": response
        }

# AzureChatOpenAI client using Mistral model
def create_mistral_model(model_name):
    """
    Creates and returns an AzureChatOpenAI model instance for GPT-3.5.

    Returns:
        AzureChatOpenAI: The configured GPT-3.5 model.
    """
    return ChatOpenAI(
        api_key=os.getenv("NETVIBES_API_KEY"),
        base_url=os.getenv("NETVIBES_BASE_URL"),
        model=os.getenv("NETVIBES_OPENAI_MISTRAL_DEPLOYMENT_NAME"),
        max_tokens=5000,
        name=model_name,
    )

# Simulated StateMemory class
class StateMemory:
    def __init__(self):
        self.requirements = {}
        self.physicals = {}

    def get_state_as_json(self):
        # Return the state as a JSON-like dictionary for display
        return {
            "requirements": {req_id: req.__dict__ for req_id, req in self.requirements.items()},
            "physicals": {phy_id: phy.__dict__ for phy_id, phy in self.physicals.items()}
        }

# Requirement and Physical classes
class Requirement:
    def __init__(self, name, description):
        self.id = str(time.time())  # Simulated unique ID
        self.name = name
        self.description = description

class Physical:
    def __init__(self, name, description, parent_id):
        self.id = str(time.time())  # Simulated unique ID
        self.name = name
        self.description = description
        self.parent_id = parent_id

# Example usage
if __name__ == "__main__":
    # Initialize the AzureChatOpenAI client with the Mistral model
    llm_client = create_mistral_model("Mistral-Model")

    # Initialize the state memory
    state_memory = StateMemory()

    # Define requirements data
    requirements_data = [
        {"index": 1, "title": "Power Source 120v", "description": "battery / any other power source of 120V"},
        {"index": 2, "title": "Motor", "description": "stepper motor"}
    ]

    # Initialize the LLM processor for parallel processing
    llm_processor_parallel = LLMProcessorParallel(llm_client, state_memory)

    # Call the function to create physicals for the given requirements in parallel
    llm_processor_parallel.create_physical_for_requirements_parallel(requirements_data)

    # Output the state as JSON
    print("\nFinal State in JSON:")
    print(state_memory.get_state_as_json())


Processing requirement 1: Power Source 120v
Processing requirement 2: Motor
Finished processing: Physical created for requirement 2: Physical for Motor
Finished processing: Physical created for requirement 1: Physical for Power Source 120v

Final State in JSON:
{'requirements': {'1737535640.0000532': {'id': '1737535640.0000532', 'name': 'Power Source 120v', 'description': 'battery / any other power source of 120V'}, '1737535640.0190554': {'id': '1737535640.0190554', 'name': 'Motor', 'description': 'stepper motor'}}, 'physicals': {'1737535651.77455': {'id': '1737535651.77455', 'name': 'Physical for Motor', 'description': AIMessage(content='**Physical Design for Stepper Motor Mounting and Control**\n\n**Objective:** To create a durable, efficient, and easy-to-mount physical design for a stepper motor, including a mounting bracket, motor shaft coupling, and an electronic control system enclosure.\n\n**Components:**\n\n1. **Stepper Motor**\n   - NEMA 17 frame (1.7" x 1.7" x 2")\n   - 200 s

# LLM -Sequential

In [10]:
import time
import os
from langchain_openai import ChatOpenAI

class LLMProcessorSequential:
    def __init__(self, llm_client, state_memory):
        self.llm_client = llm_client
        self.state_memory = state_memory

    def create_physical_for_requirements_sequential(self, requirements_data):
        for requirement in requirements_data:
            try:
                # Process each requirement one at a time
                result = self.process_requirement(requirement)
                print(f"Finished processing: {result}")
            except Exception as e:
                print(f"An error occurred while processing requirement {requirement['title']}: {e}")

    def process_requirement(self, requirement):
        index = requirement["index"]
        title = requirement["title"]
        description = requirement["description"]

        print(f"Processing requirement {index}: {title}")

        # Create a requirement object in the state memory
        req = Requirement(name=title, description=description)
        self.state_memory.requirements[req.id] = req

        # Call the LLM to create a physical for this requirement
        physical_data = self.call_llm_to_create_physical(req)

        # Create a physical object in the state memory
        physical = Physical(
            name=physical_data["name"],
            description=physical_data["description"],
            parent_id=req.id
        )
        self.state_memory.physicals[physical.id] = physical

        return f"Physical created for requirement {index}: {physical.name}"

    def call_llm_to_create_physical(self, requirement):
        """
        Calls the LLM to generate a 'physical' for the given requirement.
        """
        # Example LLM prompt
        prompt = f"Create a physical design for the following requirement in:\n\n{requirement.description}\n answer in just 10 words"

        # Call the LLM and get the response
        response = self.llm_client(prompt)

        # Example structure of the LLM response
        return {
            "name": f"Physical for {requirement.name}",
            "description": response
        }

# AzureChatOpenAI client using Mistral model
def create_mistral_model(model_name):
    """
    Creates and returns an AzureChatOpenAI model instance for GPT-3.5.

    Returns:
        AzureChatOpenAI: The configured GPT-3.5 model.
    """
    return ChatOpenAI(
        api_key=os.getenv("NETVIBES_API_KEY"),
        base_url=os.getenv("NETVIBES_BASE_URL"),
        model=os.getenv("NETVIBES_OPENAI_MISTRAL_DEPLOYMENT_NAME"),
        max_tokens=5000,
        name=model_name,
    )

# Simulated StateMemory class
class StateMemory:
    def __init__(self):
        self.requirements = {}
        self.physicals = {}

    def get_state_as_json(self):
        # Return the state as a JSON-like dictionary for display
        return {
            "requirements": {req_id: req.__dict__ for req_id, req in self.requirements.items()},
            "physicals": {phy_id: phy.__dict__ for phy_id, phy in self.physicals.items()}
        }

# Requirement and Physical classes
class Requirement:
    def __init__(self, name, description):
        self.id = str(time.time())  # Simulated unique ID
        self.name = name
        self.description = description

class Physical:
    def __init__(self, name, description, parent_id):
        self.id = str(time.time())  # Simulated unique ID
        self.name = name
        self.description = description
        self.parent_id = parent_id

# Example usage
if __name__ == "__main__":
    # Initialize the AzureChatOpenAI client with the Mistral model
    llm_client = create_mistral_model("Mistral-Model")

    # Initialize the state memory
    state_memory = StateMemory()

    # Define requirements data
    requirements_data = [
        {"index": 1, "title": "Power Source 120v", "description": "battery / any other power source of 120V"},
        {"index": 2, "title": "Motor", "description": "stepper motor"}
    ]

    # Initialize the LLM processor for sequential processing
    llm_processor_sequential = LLMProcessorSequential(llm_client, state_memory)

    # Call the function to create physicals for the given requirements sequentially
    llm_processor_sequential.create_physical_for_requirements_sequential(requirements_data)

    # Output the state as JSON
    print("\nFinal State in JSON:")
    print(state_memory.get_state_as_json())


Processing requirement 1: Power Source 120v
Finished processing: Physical created for requirement 1: Physical for Power Source 120v
Processing requirement 2: Motor
Finished processing: Physical created for requirement 2: Physical for Motor

Final State in JSON:
{'requirements': {'1737536285.2592704': {'id': '1737536285.2592704', 'name': 'Power Source 120v', 'description': 'battery / any other power source of 120V'}, '1737536302.897834': {'id': '1737536302.897834', 'name': 'Motor', 'description': 'stepper motor'}}, 'physicals': {'1737536297.997359': {'id': '1737536297.997359', 'name': 'Physical for Power Source 120v', 'description': AIMessage(content='"120V Power Source: Battery, 120V Inverter, Wall Adapter"', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 34, 'total_tokens': 56}, 'model_name': 'mistralai/Mistral-Nemo-Instruct-2407', 'system_fingerprint': '7f20ce52', 'finish_reason': 'stop', 'logprobs': None}, id='run-a09f5ec0-5642-4b31-af4f-84a385bcb694-0'

# llm parallel with immediet print

In [14]:
import concurrent.futures
import time
import os
from langchain_openai import ChatOpenAI

class LLMProcessorParallel:
    def __init__(self, llm_client, state_memory):
        self.llm_client = llm_client
        self.state_memory = state_memory

    def create_physical_for_requirements_parallel(self, requirements_data):
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Map each requirement to a separate thread
            futures = {
                executor.submit(self.process_requirement, requirement): requirement
                for requirement in requirements_data
            }

            # As each future completes, process the result and print it
            for future in concurrent.futures.as_completed(futures):
                requirement = futures[future]  # Retrieve the original requirement for context
                try:
                    result = future.result()  # Get the result
                    print(f"Finished processing for {requirement['title']}: {result}")
                except Exception as e:
                    print(f"An error occurred while processing {requirement['title']}: {e}")

    def process_requirement(self, requirement):
        index = requirement["index"]
        title = requirement["title"]
        description = requirement["description"]

        print(f"Started processing requirement {index}: {title}")

        # Create a requirement object in the state memory
        req = Requirement(name=title, description=description)
        self.state_memory.requirements[req.id] = req

        # Call the LLM to create a physical for this requirement
        physical_data = self.call_llm_to_create_physical(req)

        # Create a physical object in the state memory
        physical = Physical(
            name=physical_data["name"],
            description=physical_data["description"],
            parent_id=req.id
        )
        self.state_memory.physicals[physical.id] = physical

        return f"Physical created: {physical.name}"

    def call_llm_to_create_physical(self, requirement):
        """
        Calls the LLM to generate a 'physical' for the given requirement.
        """
        # Example LLM prompt
        prompt = f"Create a physical design for the following requirement:\n\n{requirement.description}\n. not more than 10 words for each"

        # Call the LLM and get the response
        response = self.llm_client(prompt)

        # Example structure of the LLM response
        return {
            "name": f"Physical for {requirement.name}",
            "description": response
        }

# AzureChatOpenAI client using Mistral model
def create_mistral_model(model_name):
    """
    Creates and returns an AzureChatOpenAI model instance for GPT-3.5.

    Returns:
        AzureChatOpenAI: The configured GPT-3.5 model.
    """
    return ChatOpenAI(
        api_key=os.getenv("NETVIBES_API_KEY"),
        base_url=os.getenv("NETVIBES_BASE_URL"),
        model=os.getenv("NETVIBES_OPENAI_MISTRAL_DEPLOYMENT_NAME"),
        max_tokens=5000,
        name=model_name,
    )

# Simulated StateMemory class
class StateMemory:
    def __init__(self):
        self.requirements = {}
        self.physicals = {}

    def get_state_as_json(self):
        # Return the state as a JSON-like dictionary for display
        return {
            "requirements": {req_id: req.__dict__ for req_id, req in self.requirements.items()},
            "physicals": {phy_id: phy.__dict__ for phy_id, phy in self.physicals.items()}
        }

# Requirement and Physical classes
class Requirement:
    def __init__(self, name, description):
        self.id = str(time.time())  # Simulated unique ID
        self.name = name
        self.description = description

class Physical:
    def __init__(self, name, description, parent_id):
        self.id = str(time.time())  # Simulated unique ID
        self.name = name
        self.description = description
        self.parent_id = parent_id

# Example usage
if __name__ == "__main__":
    # Initialize the AzureChatOpenAI client with the Mistral model
    llm_client = create_mistral_model("Mistral-Model")

    # Initialize the state memory
    state_memory = StateMemory()

    # Define requirements data
    requirements_data = [
        {"index": 1, "title": "Power Source 120v", "description": "battery / any other power source of 120V"},
        {"index": 2, "title": "Motor", "description": "stepper motor"},
        {"index": 3, "title": "Control System", "description": "embedded system with sensors"}
    ]

    # Initialize the LLM processor for parallel processing
    llm_processor_parallel = LLMProcessorParallel(llm_client, state_memory)

    # Call the function to create physicals for the given requirements in parallel
    llm_processor_parallel.create_physical_for_requirements_parallel(requirements_data)

    # Output the state as JSON
    print("\nFinal State in JSON:")
    print(state_memory.get_state_as_json())


Started processing requirement 1: Power Source 120v
Started processing requirement 2: Motor
Started processing requirement 3: Control System
Finished processing for Power Source 120v: Physical created: Physical for Power Source 120v
Finished processing for Control System: Physical created: Physical for Control System
Finished processing for Motor: Physical created: Physical for Motor

Final State in JSON:
{'requirements': {'1737538306.8941405': {'id': '1737538306.8941405', 'name': 'Power Source 120v', 'description': 'battery / any other power source of 120V'}, '1737538306.9191465': {'id': '1737538306.9191465', 'name': 'Motor', 'description': 'stepper motor'}, '1737538306.9401474': {'id': '1737538306.9401474', 'name': 'Control System', 'description': 'embedded system with sensors'}}, 'physicals': {'1737538317.4852731': {'id': '1737538317.4852731', 'name': 'Physical for Power Source 120v', 'description': AIMessage(content="Here's a simple physical design concept that meets the given requ

In [19]:
import concurrent.futures
import time
import os
from langchain_openai import ChatOpenAI


class LLMProcessorParallel:
    def __init__(self, llm_client, state_memory):
        self.llm_client = llm_client
        self.state_memory = state_memory

    def process_requirements_parallel(self):
        # Retrieve all requirements from state_memory
        requirements = list(self.state_memory.requirements.values())

        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Map each requirement to a separate thread
            futures = [
                executor.submit(self.process_requirement, requirement)
                for requirement in requirements
            ]

            # Handle each future as it completes
            for future in concurrent.futures.as_completed(futures):
                try:
                    # Get the result and print it
                    result = future.result()
                    print(result)
                except Exception as e:
                    print(f"An error occurred: {e}")

    def process_requirement(self, requirement):
        """
        Process a single requirement: call LLM, add to state_memory, retrieve, and print.
        """
        # Call the LLM to create a physical for the requirement
        physical_data = self.call_llm_to_create_physical(requirement)

        # Create a physical object and add it to state_memory
        physical = Physical(
            name=physical_data["name"],
            description=physical_data["description"],
            parent_id=requirement.id
        )
        self.state_memory.physicals[physical.id] = physical

        # Retrieve the created physical from state_memory
        created_physical = self.state_memory.physicals[physical.id]

        # Return the result to be printed
        return f" Physical created: {created_physical.name} for requirement {requirement.name}, 'description' : {created_physical.description.content}"

    def call_llm_to_create_physical(self, requirement):
        """
        Calls the LLM to generate a 'physical' for the given requirement.
        """
        # Example LLM prompt
        prompt = f"Create a physical design for the following requirement:\n\n{requirement.description}\n. each should be not more than 10 words"

        # Call the LLM and get the response
        response = self.llm_client(prompt)

        # Example structure of the LLM response
        return {
            "name": f"Physical for {requirement.name}",
            "description": response
        }


# AzureChatOpenAI client using Mistral model
def create_mistral_model(model_name):
    """
    Creates and returns an AzureChatOpenAI model instance for GPT-3.5.

    Returns:
        AzureChatOpenAI: The configured GPT-3.5 model.
    """
    return ChatOpenAI(
        api_key=os.getenv("NETVIBES_API_KEY"),
        base_url=os.getenv("NETVIBES_BASE_URL"),
        model=os.getenv("NETVIBES_OPENAI_MISTRAL_DEPLOYMENT_NAME"),
        max_tokens=5000,
        name=model_name,
    )


# Simulated StateMemory class
class StateMemory:
    def __init__(self):
        self.requirements = {}
        self.physicals = {}

    def get_state_as_json(self):
        # Return the state as a JSON-like dictionary for display
        return {
            "requirements": {req_id: req.__dict__ for req_id, req in self.requirements.items()},
            "physicals": {phy_id: phy.__dict__ for phy_id, phy in self.physicals.items()}
        }


# Requirement and Physical classes
class Requirement:
    def __init__(self, name, description):
        self.id = str(time.time())  # Simulated unique ID
        self.name = name
        self.description = description


class Physical:
    def __init__(self, name, description, parent_id):
        self.id = str(time.time())  # Simulated unique ID
        self.name = name
        self.description = description
        self.parent_id = parent_id


# Example usage
if __name__ == "__main__":
    # Initialize the AzureChatOpenAI client with the Mistral model
    llm_client = create_mistral_model("Mistral-Model")

    # Initialize the state memory
    state_memory = StateMemory()

    # Add example requirements to state memory
    state_memory.requirements["1"] = Requirement(name="Power Source 120v", description="battery / any other power source of 120V")
    state_memory.requirements["2"] = Requirement(name="Motor", description="stepper motor")
    state_memory.requirements["3"] = Requirement(name="Control System", description="embedded system with sensors")

    # Initialize the LLM processor
    llm_processor_parallel = LLMProcessorParallel(llm_client, state_memory)

    # Process requirements in parallel
    llm_processor_parallel.process_requirements_parallel()

    # Output the final state as JSON
    print("\nFinal State in JSON:")
    print(state_memory.get_state_as_json())


 Physical created: Physical for Motor for requirement Motor, 'description' : **Stepper Motor Physical Design**

- Mounting Hole: 2 x 3mm
- Stepper Motor: 23mm (Diameter) x 32mm (Length)
- Shaft: 5mm (Diameter)
- Wiring: 4 x 24AWG (2mm spacing)
 Physical created: Physical for Control System for requirement Control System, 'description' : **Embedded System Design: Sensor Node**

1. **Microcontroller**: ESP32-WROOM-32 (Low-power, dual-core, Wi-Fi/BT)
2. **Power Supply**: CR2032 coin cell battery
3. **Sensors**:
   - DHT22 (Temperature & Humidity)
   - BH1750 (Light Intensity)
   - SR04 (Ultrasonic Distance)
4. **Connectivity**: Wi-Fi (For data transmission)
5. **Physical Layout**:
   - PCB dimensions: 50mm x 30mm
   - Sensors aligned along the edges for optimal measurement
   - Battery placed at the center for even weight distribution
   - Microcontroller positioned for minimal routing and efficient heat dissipation
6. **Enclosure**:
   - Plastic case (e.g., ABS) with openings for sensors